# Landcover Classification Example

#### Sources: 
- https://blog.gishub.org/earth-engine-tutorial-32-machine-learning-with-earth-engine-supervised-classification
- https://geohackweek.github.io/GoogleEarthEngine/05-classify-imagery/
- https://ceholden.github.io/open-geo-tutorial/python/chapter_5_classification.html
- GEE Documentation

#### Steps:
1. Collect training data. Assemble features which have a property that stores the known class label and properties storing numeric values for the predictors.
2. Instantiate a classifier. Set its parameters if necessary.
3. Train the classifier using the training data.
4. Classify an image or feature collection.
5. Estimate classification error with independent validation data.

The training data is a `FeatureCollection` with a property storing the class label and properties storing predictor variables. Class labels should be consecutive, integers starting from 0. If necessary, use remap() to convert class values to consecutive integers. The predictors should be numeric.

### Import libraries

In [1]:
import ee
import geemap
from geemap import *
import json
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON
import os
import sklearn
# !pip install geemap


## Data Preparation

### Create an interactive map

In [2]:
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

### Add region data to the map

In [5]:
point = ee.Geometry.Point(-122.4439, 37.7538)

#making a cloud free Landsat 8 Surface Reflectance Composite
image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(point).filterDate('2016-01-01', '2016-12-31').sort('CLOUD_COVER').first()

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'] #True Colour Composite
}


Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map
#Map.addLayer(aoi, "Mai_Ndombe");


Map(bottom=1614465.046875, center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['…

### Mask the image for clouds

In [4]:
'''
#cloud mask
def cloudMask(image):
  #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
    #Get the pixel QA band.
    qa = image.select('pixel_qa');
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0));
    return image.updateMask(mask);

image = cloudMask(image)
Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8_Masked")
Map
'''

Map(center=[37.75379999999999, -122.44390000000001], controls=(WidgetControl(options=['position', 'transparent…

### Check image properties

In [6]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2016-11-18'

In [7]:
image.get('CLOUD_COVER').getInfo()

0.08

### Creating the training dataset

There are several ways you can create a region for generating the training dataset.

- Draw a shape (e.g., rectangle) on the map and the use `region = Map.user_roi`
- Define a geometry, such as `region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])`
- Create a buffer zone around a point, such as `region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)`
- If you don't define a region, it will use the image footprint by default

In [8]:
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
region = ee.Geometry.Point([-122.4439, 37.7538]).buffer(10000)

The [USGS National Land Cover Database (NLCD)](https://developers.google.com/earth-engine/datasets/catalog/USGS_NLCD) will be used to create label dataset for training


![](https://i.imgur.com/7QoRXxu.png)

In [19]:
nlcd = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image.geometry()) #pre-defined data from an Earth Engine table asset
#remapping the LULC categories to new values ranging from 0–19 to make 
#visualizations consistent across products later in the script.

#remapping class values for nlcd data
class_values = nlcd.get('landcover_class_values')
class_values_remap = [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19]

nlcd = nlcd.remap(class_values,
         class_values_remap).rename('Landcover_Class')

#import the colour palette of the nlcd data
classColours = (
  '476ba1','d1defa','decaca','d99482','ee0000',
  'ab0000','b3aea3','68ab63','1c6330','b5ca8f',
  'a68c30','ccba7d','e3e3c2','caca78','99c247',
  '78ae94','dcd93d','ab7028','bad9eb','70a3ba'
    )

Map.addLayer(nlcd, {'min':0, 'max':19, 'palette':classColours}, 'NLCD')
Map


Map(bottom=25521.0, center=[38.25112269630296, -122.51678466796876], controls=(WidgetControl(options=['positio…

In [ ]:
# Make the training dataset.
'''points = nlcd.sample(**{
    'region': image.geometry(), #The region to sample from. If unspecified, uses the image's whole footprint.
    'scale': 30, #A nominal scale in meters of the projection to sample in.
    'numPixels': 5000, #The approximate number of pixels to sample.
    'seed': 0, #A randomization seed to use for subsampling.
    'geometries': True  # If true, adds the center of the sampled pixel as the geometry property of the output 
                        #feature. Otherwise, geometries will be omitted (saving memory).Set this to False to 
                        #ignore geometries
})
'''
sample = nlcd.sample(**{
    'region': image.geometry(), #The region to sample from. If unspecified, uses the image's whole footprint.
    'scale': 30, #A nominal scale in meters of the projection to sample in.
    'numPixels': 50000, #The approximate number of pixels to sample.
    'seed': 0, #A randomization seed to use for subsampling.
    'geometries': True  # If true, adds the center of the sampled pixel as the geometry property of the output 
                        #feature. Otherwise, geometries will be omitted (saving memory).Set this to False to 
                        #ignore geometries
})


#The randomColumn() method will add a column of uniform random
#numbers in a column named 'random' by default.
sample = sample.randomColumn();

split = 0.7;  #Roughly 70% training, 30% testing.
training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

Map.addLayer(training, {}, 'Training')
Map

In [ ]:
print(training.size().getInfo())

In [ ]:
print(training.first().getInfo()) #Returns the first entry from a given collection.

In [ ]:
#Export the training data as csv
geemap.ee_to_csv(training, 'lc_training_data.csv')


In [ ]:
print(validation.size().getInfo())

In [ ]:
#Export the validation data as csv
geemap.ee_to_csv(validation, 'lc_validation_data.csv')


### Train the classifier

Here we complete supervised classification using the RandomForest ensemble decision tree algorithm by [Leo Breiman and Adele Cutler](https://link.springer.com/article/10.1023/A:1010933404324).

The RandomForest algorithm is popular in the field of remote sensing, and is quite fast compared to some other machine learning approaches (e.g., SVM can be quite computationally intensive). It isn't necessarily the best but provides great first step into the world of machine learning for classification and regression.

In [ ]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'Landcover_Class'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': training,
  'properties': [label],
  'scale': 30
})

# Train a Random Forest classifier with 10 decision trees (will employ hyperparameter testing outside of notebook)
classifier = ee.Classifier.smileRandomForest(100).train(training,label,bands)

In [ ]:
print(training.first().getInfo())

In [ ]:
#Export the data as csv
geemap.ee_to_csv(training, 'trained_data.csv')


### Classify the image

In [ ]:
# Classify the image with the same bands used for training.
classified = image.select(bands).classify(classifier)

# Display the clusters with random colors.
Map.addLayer(classified.randomVisualizer(), {}, 'classfied')
Map

### Render categorical map

To render a categorical map, we can set two image properties: `landcover_class_values` and `landcover_class_palette`. We can use the same style as the NLCD so that it is easy to compare the two maps. 

In [ ]:
classValues = class_values_remap 
classValues

In [ ]:
class_palette = nlcd.get('landcover_class_palette').getInfo()
class_palette

In [ ]:
landcover = classified.set('classification_class_values', classValues)
landcover = landcover.set('classification_class_palette', class_palette)

In [ ]:
Map.addLayer(landcover, {}, 'Land Cover Map')
Map

### Visualize the result

In [ ]:
print('Change layer opacity:')
cluster_layer = Map.layers[-1]
cluster_layer.interact(opacity=(0, 1, 0.1))

### Add a legend to the map

In [ ]:
Map.add_legend(builtin_legend='NLCD')
Map

### Export the result



In [ ]:
#tif file
geemap.ee_export_image(landcover, 'Land_Cover_Classification.tif', scale=900)

### Assess The Accuracy

A confusion matrix is used here to compare two different image grids. We will be using it primarily to compute overall accuracy between the model and the validation layer, but confusion matrices also provide explicit information about which LULC classes were classified incorrectly; not just if pixels were classified incorrectly, but what LULC they were incorrectly classified as. 

In [ ]:
#Get a confusion matrix representing resubstitution accuracy.
train_conf_matrix = classifier.confusionMatrix()
train_conf_matrix.getInfo()


In [ ]:
#Get the overall train accuracy
train_accuracy = train_conf_matrix.accuracy()
train_accuracy.getInfo()

The accuracy estimated from training data is an overestimate because the random forest is “fit” to the training data.

To get validation accuracy, we classify the validation data.

We want to ensure that the training samples are uncorrelated with the evaluation samples. This might result from spatial autocorrelation of the phenomenon being predicted. One way to exclude samples that might be correlated in this manner is to remove samples that are within some distance to any other sample(s). This can be accomplished with a spatial join:

### Validation on NLCD Data

In [ ]:
# Overlay the points on the imagery to get validation points.
validation = image.select(bands).sampleRegions(**{
  'collection': validation,
  'properties': [label],
  'scale': 30
})

In [ ]:
print(validation.first().getInfo())

In [ ]:
validated = validation.classify(classifier)

In [ ]:
#Export the data as csv
geemap.ee_to_csv(validated, 'validated_data.csv')

In [ ]:
#Get a confusion matrix representing resubstitution accuracy.
#errorMatrix computes a 2D error matrix for a collection by comparing two columns 
#of a collection: one containing the actual values, and one containing predicted values.
test_conf_matrix = validated.errorMatrix('Landcover_Class', 'classification')
test_conf_matrix.getInfo()


In [ ]:
#Get the overall test accuracy
test_accuracy = test_conf_matrix.accuracy()
test_accuracy.getInfo()

The model does not perform too well on data it hasn't seen before. Hyperparameter testing will need to be performed to improve this performance.

Already just adjusting from 10 to 100 trees moved the accuracy from 0.58 to 0.62.

## Validation on AOI (Mai Ndombe)

#### Set-Up Data for first time period for AOI

In [ ]:
'''
#clip image to only include aoi region
file_path = os.path.abspath('/Users/joycelynlongdon/Desktop/Cambridge/CambridgeCoding/MRES/GEE_examples/sample.json')

with open(file_path) as f:
    aoi_poly = json.load(f)

geometry = ee.FeatureCollection(aoi_poly).first().geometry()
'''


#### Create a cloud free Landsat 8 Surface Reflectance Composite


In [ ]:
'''
point = ee.Geometry.Point(18.3300, -2.00)
#pointBuffer = point.buffer(100000)
image_aoi = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate('2016-06-01', '2016-08-31').sort('CLOUD_COVER').first().clip(geometry)

def cloudMask(image):
  #Bits 3 and 5 are cloud shadow and cloud, respectively.
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
    #Get the pixel QA band.
    qa = image_aoi.select('pixel_qa');
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0));
    return image_aoi.updateMask(mask);


image_aoi = cloudMask(image_aoi)
#taking out any remaining cloud cover with the bitmask QA band
#qa = image.select('pixel_qa')
#cloudMask = qa.bitwiseAnd(1<<5).eq(0)
#.and(qa.bitwiseAnd(1<<3).eq(0))
#masked = image.updateMask(cloudMask).clip(bounds)

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 8)
Map.addLayer(image_aoi, vis_params, "Landsat-8")
Map
'''

In [ ]:
'''
aoi_class = image_aoi.classify(classifier)
geemap.ee_to_csv(aoi_class, 'aoi_data.csv')
'''

In [ ]:
'''
Map.addLayer(aoi_class, vis_params, "AOI Classification")
'''